In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataset = pd.read_csv('database_gmal1801.csv')

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dataset.dropna (axis=1, how='all', inplace=True)
dataset.replace ('None', 0, regex=True, inplace=True)
dataset.drop (columns=['Unnamed: 0', 'assettypenumber', 'primaryloanservicername'], inplace=True)

In [4]:
dataset.head()

,assetnumber,reportingperiodbeginningdate,reportingperiodendingdate,originatorname,originationdate,originalloanamount,originalloanterm,loanmaturitydate,originalinterestratepercentage,interestcalculationtypecode,...,zerobalanceeffectivedate,zerobalancecode,currentdelinquencystatus,assetsubjectdemandindicator,chargedoffprincipalamount,recoveredamount,modificationtypecode,paymentextendednumber,repossessedindicator,repossessedproceedsamount
0,0001724128 - 000001,07-01-2019,07-31-2019,GM FINANCIAL,11/2015,16477.10,73,03/2022,0.0645,1,...,NaN,NaN,7.0,False,0.0,0.0,NaN,0,False,0.0
1,0001724128 - 000002,07-01-2019,07-31-2019,GM FINANCIAL,11/2015,25507.47,76,10/2022,0.0280,1,...,NaN,NaN,0.0,False,0.0,0.0,NaN,0,False,0.0
2,0001724128 - 000005,07-01-2019,07-31-2019,GM FINANCIAL,11/2015,42746.64,61,03/2021,0.0665,1,...,NaN,NaN,11.0,False,0.0,0.0,NaN,0,False,0.0
3,0001724128 - 000008,07-01-2019,07-31-2019,GM FINANCIAL,11/2015,24377.86,74,04/2022,0.1080,1,...,NaN,NaN,88.0,False,0.0,0.0,NaN,0,False,0.0
4,0001724128 - 000009,07-01-2019,07-31-2019,GM FINANCIAL,11/2015,49155.07,74,08/2022,0.0625,1,...,NaN,NaN,16.0,False,0.0,0.0,NaN,0,False,0.0


In [5]:
dataset.shape

(1021633, 64)

In [6]:
var_factors = ['interestcalculationtypecode', 'interestcalculationtypecode', 'underwritingindicator', 'paymenttypecode',
               'subvented', 'vehiclenewusedcode', 'vehicletypecode', 'vehiclevaluesourcecode',
               'obligoremploymentverificationcode',
               'obligorincomeverificationlevelcode', 'coobligorindicator', 'obligorgeographiclocation',
               'assetaddedindicator',
               'reportingperiodmodificationindicator', 'servicingadvancemethodcode', 'assetsubjectdemandindicator',
               'repossessedindicator'] #'zerobalancecode'
var_num = ['originalloanamount', 'originalloanterm', 'originalinterestratepercentage', 'vehiclevalueamount',
              'obligorcreditscore',
              'paymenttoincomepercentage', 'remainingtermtomaturitynumber', 'reportingperiodbeginningloanbalanceamount',
              'nextreportingperiodpaymentamountdue',
              'reportingperiodinterestratepercentage', 'nextinterestratepercentage', 'servicingfeepercentage',
              'servicingflatfeeamount',
              'otherservicerfeeretainedbyservicer', 'otherassesseduncollectedservicerfeeamount',
              'scheduledinterestamount', 'scheduledprincipalamount',
              'otherprincipaladjustmentamount', 'reportingperiodactualendbalanceamount',
              'reportingperiodscheduledpaymentamount', 'totalactualamountpaid',
              'actualinterestcollectedamount', 'actualprincipalcollectedamount', 'actualothercollectedamount',
              'serviceradvancedamount',
              'chargedoffprincipalamount', 'recoveredamount', 'repossessedproceedsamount']
var = var_factors + var_num
# Get all the column index for independent variables
column_num = [dataset.columns.get_loc (i) for i in var]
# Creat dependent variables
dataset['Delinquency'] = dataset.loc[:, 'currentdelinquencystatus'] != 0
var_drop = list(set(dataset.columns.values)-set(var))
dataset = pd.get_dummies(dataset, columns=var_factors)

In [7]:
X = dataset.drop (columns=var_drop)
y = dataset['Delinquency']

In [9]:
!pip install missingpy

  Using cached https://files.pythonhosted.org/packages/b5/be/998d04d27054b58f0974b5f09f8457778a0a72d4355e0b7ae877b6cfb850/missingpy-0.2.0-py3-none-any.whl


In [11]:
from missingpy import MissForest
imputer = MissForest()
X_impute = imputer.fit_transform(X)

Iteration: 0
Iteration: 1
Iteration: 2


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_validation = X_test.values
y_train = y_train.values
y_validation = y_test.values

In [ ]:
print('X_shapes:\n', 'X_train:', 'X_validation:\n', X_train.shape, X_validation.shape, '\n')
print('Y_shapes:\n', 'Y_train:', 'Y_validation:\n', y_train.shape, y_validation.shape)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [ ]:
##Spot-Checking Algorithms
%timeit
models = []

models.append(('LR', LogisticRegression(n_jobs=6)))
models.append(('LDA', LinearDiscriminantAnalysis(n_jobs=6)))
models.append(('CART', DecisionTreeClassifier(n_jobs=6)))
models.append(('XGB', XGBClassifier(n_jobs=6)))
models.append(('RF', RandomForestClassifier(n_jobs=6)))
models.append(('SVM', LinearSVC(n_jobs=6)))

#testing models

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=5)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
    results.append(cv_results)
    names.append(name)
    msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())